<a href="https://colab.research.google.com/github/insooAI/My-Portfolio/blob/main/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB 영화평 감성분석

In [ ]:
train = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip',delimiter='\t')
test = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip',delimiter='\t')

In [ ]:
display(train,test)

## Data Preprocessing

In [ ]:
import re
def cleanhtml(raw):
    cleanr = re.compile('<br />')
    cleantext = re.sub(cleanr, '', raw)
    return cleantext

def etcRemove(x):
    return re.sub('[^0-9a-zA-Z${3}#{3}| |,|.|!|?]', "",x)

def tolower(x) :         # 소문자로 통일하는 함수                                                                                                                                                                                                                                                                                                                                                                           
    try :                                                                                                                                                                                                                                                                                                                                                                                     
        return x.lower()                                                                                                                                                                                                                                                                                                                                                                      
    except :                                                                                                                                                                                                                                                                                                                                                                                  
        return ""

def prep(x):
    return tolower(etcRemove(cleanhtml(x)))

In [ ]:
alldata = pd.concat([train,test])
alldata

In [ ]:
# 데이터 전처리
alldata['review'] = list(prep(x) for x in alldata['review'])

In [ ]:
alldata2 = alldata.drop(['id','sentiment'],1)
target = train['sentiment']
alldata2

In [ ]:
# 확인 차
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

## 모델 선택


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
tfidf_vect = TfidfVectorizer()
lr_clf = LogisticRegression()

# tf-idf & 로지스틱
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=1000)),
    ('lr_clf', LogisticRegression(C=10, random_state=12))
])
pipeline.fit(train2['review'], target)
result = pipeline.predict_proba(test2['review'])
print(result)
# print('pipeline을 통한 Logistic Regressiond의 예측 정확도는 {0:.3f}'.format(accuracy_score(target, pred)))

In [ ]:
sub = pd.DataFrame(data=result[:,1],columns=['sentiment'])
sub["id"] = test["id"]
sub.set_index("id",inplace=True)

sub.to_csv('sub.csv')


In [ ]:
sub